<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llama31groqneo4jKV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
#https://console.groq.com/keys - register and get GROQ_API_KEY key
#https://console.groq.com/docs/libraries - Groq documentation
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v1/
#https://github.com/projectwilsen/KnowledgeGraphLLM/blob/main/tutorial/2_convert_any_text_into_a_kg/notebook.ipynb
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook_groq/
#https://www.youtube.com/watch?v=ky8LQE-82xs
#https://www.youtube.com/watch?v=KMXQ4SVLwmo

#https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/

#https://docs.llamaindex.ai/en/stable/understanding/loading/loading/
#https://docs.llamaindex.ai/en/stable/module_guides/indexing/lpg_index_guide/
#https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/settings/
#https://docs.llamaindex.ai/en/stable/examples/llm/groq/
#https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe.txt

#https://console.neo4j.io/

!pip install groq
!pip install neo4j
!pip install llama-index
!pip install llama-index.core
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-graph-stores-nebula
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llm-ollama
!pip install llama-index-llms-groq
!pip install nest_asyncio

ERROR: Could not find a version that satisfies the requirement llama-index-llm-ollama (from versions: none)
ERROR: No matching distribution found for llama-index-llm-ollama


In [76]:
!pip show llama-index

Name: llama-index
Version: 0.10.61
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 


In [69]:
!mkdir -p 'data/receipekv2'
!mkdir -p 'data/PerPosOrgv3'
!wget 'https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe.txt' -O 'data/receipekv2/receipekv2.txt'
!wget 'https://github.com/vkrisvasan/llamaKV/blob/main/PersonPositionOrg.txt' -O 'data/PerPosOrgv3/PerPosOrgkv2.txt'
#!wget 'https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe4.txt' -O 'data/receipekv2/receipekv2.txt'
#!wget 'https://gutenberg.org/cache/epub/65061/pg65061.txt' -O 'data/receipekv1/receipekv11.txt'

--2024-08-06 02:38:55--  https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/receipekv2/receipekv2.txt’

data/receipekv2/rec     [ <=>                ] 163.20K  --.-KB/s    in 0.05s   

2024-08-06 02:38:56 (3.13 MB/s) - ‘data/receipekv2/receipekv2.txt’ saved [167119]

--2024-08-06 02:38:56--  https://github.com/vkrisvasan/llamaKV/blob/main/PersonPositionOrg.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/PerPosOrgv3/PerPosOrgkv2.txt’

data/PerPosOrgv3/Pe     [ <=>                ] 162.58K  --.-KB/s    in 0.05s   

2024-08-06 02:38:56 (3.22 MB/s) - ‘data/PerPosOrgv3/PerPosOrgkv2.txt’ saved [166

In [70]:

from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/PerPosOrgv3/").load_data()


In [71]:
import os
import getpass
credential_names = ["GROQ_API_KEY","NEO4J_CONNECTION_URL","NEO4J_USER","NEO4J_PASSWORD","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [72]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username=os.environ["NEO4J_USER"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_CONNECTION_URL"],
)

In [73]:
from llama_index.core.graph_stores.types import EntityNode, ChunkNode, Relation

# Create a two entity nodes
entity1 = EntityNode(label="PERSON", name="Logan", properties={"age": 28})
entity2 = EntityNode(label="ORGANIZATION", name="LlamaIndex")

# Create a relation
relation = Relation(
    label="WORKS_FOR",
    source_id=entity1.id,
    target_id=entity2.id,
    properties={"since": 2023},
)

In [74]:
graph_store.upsert_nodes([entity1, entity2])
graph_store.upsert_relations([relation])

In [75]:
from llama_index.core.schema import TextNode

source_node = TextNode(text="Logan (age 28), works for LlamaIndex since 2023.")
relations = [
    Relation(
        label="MENTIONS",
        target_id=entity1.id,
        source_id=source_node.node_id,
    ),
    Relation(
        label="MENTIONS",
        target_id=entity2.id,
        source_id=source_node.node_id,
    ),
]

graph_store.upsert_llama_nodes([source_node])
graph_store.upsert_relations(relations)

In [77]:

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model="llama-3.1-8b-instant",api_key=os.environ["GROQ_API_KEY"])

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [78]:
print(type(Settings.llm))

<class 'llama_index.llms.groq.base.Groq'>


In [81]:
import os
import nest_asyncio
import asyncio
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

nest_asyncio.apply()

# Define an async function to create the PropertyGraphIndex
async def create_index():
    try:
        index = await PropertyGraphIndex.from_documents(
            documents,
            embed_model=Settings.embed_model,
            kg_extractors=[
                SchemaLLMPathExtractor(
                    llm=Settings.llm
                )
            ],
            property_graph_store=graph_store,
            show_progress=True,
        )
        return index
    except Exception as e:
        print(f"An error occurred: {e}")
        if hasattr(e, 'response'):
            print(f"Error response: {e.response}")
        if hasattr(e, 'args'):
            print(f"Error args: {e.args}")
            # Attempt to print more specific details if available
            if isinstance(e.args, tuple) and len(e.args) > 0:
                error_detail = e.args[0]
                print(f"Error detail: {error_detail}")

In [82]:
# Run the async function in the current event loop
index = asyncio.get_event_loop().run_until_complete(create_index())

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]








Extracting paths from text with schema:   0%|          | 0/83 [00:00<?, ?it/s]

An error occurred: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "KGSchema"\n      },\n      "parameters": {\n        "triplets": [\n          {\n            "subject": "file_path",\n            "relation": "contains",\n            "object": "/content/data/PerPosOrgv3/PerPosOrgkv2.txt"\n          },\n          {\n            "subject": "/content/data/PerPosOrgv3/PerPosOrgkv2.txt",\n            "relation": "has",\n            "object": "svg"\n          },\n          {\n            "subject": "svg",\n            "relation": "has",\n            "object": "path"\n          },\n          {\n            "subject": "path",\n            "relation": "has",\n            "object": "M0 1.75A.75.75 0

In [83]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor


index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=Settings.llm,
            possible_entities=["DISH", "Ingredients", "Instructions"],
            possible_relations=["CONTAINS", "COOKED_WITH", "MAKES_UP"]
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

RuntimeError: error checking inheritance of ['DISH', 'Ingredients', 'Instructions'] (type: list)

In [ ]:

from llama_index.core import SchemaLLMPathExtractor

entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
  llm=Settings.llm,
  possible_entities=entities,
  possible_relations=relations,
  kg_validation_schema=schema,
  strict=True,  # if false, allows values outside of spec
)

In [103]:
def generate_content(prompt):
  reponse = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return reponse.choices[0].message.content

In [ ]:
output = generate_content("with example can you explain SOLID design principle")
print(output)